# Exercise: Nonlinear Programming¶

In [1]:
import os
import numpy as np
from numpy.testing import assert_array_equal, assert_array_almost_equal
import matplotlib.pyplot as plt
from casadi import (
    DM, SX, MX, Function,
    vertcat, horzcat,
    norm_2, sin, cos, pi,
    integrator, rootfinder
)

import casadi
casadi.__version__

'3.6.4'

In [2]:
plot_dir = 'plots'
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)

## Golf Ball Problem

\begin{aligned}
\cases{\begin{matrix}
    \dot{\mathbf{p}} = \mathbf{v} \\
    \dot{\mathbf{v}} = \mathbf{g} - \frac{1}{m} c \| \mathbf{v} \|_2 \mathbf{v}
\end{matrix}
}
\end{aligned}

where

\begin{aligned}
\mathbf{p} = \left[ \begin{matrix} x \\ y \end{matrix} \right],
\mathbf{v} = \left[ \begin{matrix} v_x \\ v_y \end{matrix} \right],
\mathbf{g} = \left[ \begin{matrix} 0 \\ -g \end{matrix} \right]
\end{aligned}

## 1. Integrating the golf ball’s state

### 1.1 CasADi expression for the ODE

In [3]:
p = MX.sym('p', 2)
v = MX.sym('v', 2)

# Constants
m = DM(0.04593)
g = SX(2, 1)
g[1] = -9.81
c = DM(17.5e-5)
g = DM(2, 1)
g[1] = -9.81
g

DM([00, -9.81])

In [4]:
rhs = vertcat(
    v,
    g - c * norm_2(v) * v / m
)

assert rhs.shape == (4, 1)
rhs

MX(vertcat(v, ([0, -9.81]-(((0.000175*||v||_F)*v)/0.04593))))

In [5]:
f = Function('rhs', [vertcat(p, v)], [rhs])
assert_array_almost_equal(
    f([0.0, 0.0, 35.0, 30.0]), 
    [[35], [30], [-6.14737], [-15.0792]], 
    decimal=4
)

### 2. Create a CasADi integrator

In [6]:
ode = {'x': vertcat(p, v), 'ode': rhs}
t0, t1 = 0., 1.
intg = integrator('intg', 'cvodes', ode, t0, t1)
intg

Function(intg:(x0[4],z0[0],p[0],u[0],adj_xf[],adj_zf[],adj_qf[])->(xf[4],zf[0],qf[0],adj_x0[],adj_z0[],adj_p[],adj_u[]) CvodesInterface)

In [7]:
x0 = [0.0, 0.0, 35.0, 30.0]
xf, _, _, _, _, _, _ = intg(x0, [], [], [], [], [], [])
xf

DM([32.3549, 23.0661, 30.075, 16.6423])

In [8]:
assert_array_almost_equal(xf, [[32.3549], [23.0661], [30.075], [16.6423]], decimal=4)

## 2. Determining the reach of the golf ball

### 2.1 Function to compute state in one second

In [9]:
X0 = MX.sym('X0', 4)
fly1sec = Function('fly1sec', [X0], [intg(x0, [], [], [], [], [], [])[0]])
# Or fly1sec = Function('fly1sec', [X0], [intg(x0=x0)['xf']])
x1 = fly1sec([0.0,0.0,35.0,20.0])
x1

DM([32.3549, 23.0661, 30.075, 16.6423])

In [10]:
assert_array_almost_equal(x1, [[32.3549], [23.0661], [30.075], [16.6423]], decimal=4)

### 2.2 Function to compute state at time $t$



In [11]:
T = MX.sym('T')
ode_T = {'x': vertcat(p, v), 'ode': T * rhs,'p':T}
intg = integrator('intg', 'cvodes', ode_T, 0., 1.)
fly = Function('fly', [X0, T], [intg(x0=X0, p=T)['xf']])
fly

Function(fly:(i0[4],i1)->(o0[4]) MXFunction)

In [12]:
xT = fly([0.0, 0.0, 35.0, 30.0], 5)
xT

DM([130.338, 8.27205, 19.8961, -21.2868])

In [13]:
assert_array_almost_equal(xT, [[130.3377], [8.27205], [19.8961], [-21.2868]], decimal=4)

### 2.3 Shoot function

In [14]:
v = MX.sym('v')
theta = MX.sym('theta')
theta_rad = theta * pi / 180
shoot = Function(
    'shoot', 
    [v, theta, T], 
    [fly(vertcat(0., 0., v * cos(theta_rad), v * sin(theta_rad)), T)]
)
xT = shoot(50, 30, 5)
xT

DM([155.243, -11.0833, 22.6012, -23.8282])

In [15]:
assert_array_almost_equal(xT, [[155.2433], [-11.0833], [22.6012], [-23.8282]], decimal=4)